<a href="https://colab.research.google.com/github/ykitaguchi77/ROP_AI_project/blob/master/Inference_for_cvat_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# CUDAが使えるかどうかを確認
cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")


CUDA is available! GPU: Tesla T4


In [ ]:
# prompt: gdrive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#**Video2Img**

In [ ]:
import cv2
import os
from tqdm import tqdm

def extract_frames(video_path, output_dir):
    # 出力ディレクトリの作成
    os.makedirs(output_dir, exist_ok=True)

    # 動画の読み込み
    cap = cv2.VideoCapture(video_path)

    # 総フレーム数の取得
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # tqdmで進捗バーを表示
    with tqdm(total=total_frames, desc='フレーム抽出中') as pbar:
        frame_count = 0

        while True:
            # フレームの読み込み
            ret, frame = cap.read()

            # 読み込みが終了したらループを抜ける
            if not ret:
                break

            # フレームを保存
            output_path = os.path.join(output_dir, f'{os.path.basename(output_dir)}_{frame_count:04d}.jpg')
            cv2.imwrite(output_path, frame)

            frame_count += 1
            pbar.update(1)

    # キャプチャの解放
    cap.release()

    print(f'\n合計 {frame_count} フレームを抽出しました')

def main():
    # 入力動画のパス
    video_path = r'/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_movie/IMG_1703.mp4'

    # 出力ディレクトリ
    output_dir = r'/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_images/IMG_1703'

    # フレーム抽出の実行
    extract_frames(video_path, output_dir)

if __name__ == '__main__':
    main()

フレーム抽出中: 100%|██████████| 888/888 [00:31<00:00, 28.01it/s]


合計 888 フレームを抽出しました


#**YOLO_inference**

In [ ]:
!pip install ultralytics --q
!pip install wget --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# **1枚画像**

In [ ]:

from ultralytics import RTDETR
import wget
import cv2
import matplotlib.pyplot as plt

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model = RTDETR(r"/content/drive/MyDrive/Deep_learning/ROP_AI_project/models/rtdetr-l-1700_1703_1000epochs.pt")

# 推論を実行: YOLOの推論
image_path = r'/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_images/IMG_1703/IMG_1703_0024.jpg'
results = model(image_path, save=False)

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for box in boxes:
        cls_id = int(box.cls[0])  # クラスID
        x_center, y_center, width, height = box.xywhn[0]

        # YOLO形式: class x_center y_center width height
        # 小数点以下6桁で出力したい場合
        print(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    #検出結果を画像に描画して表示
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 結果の保存
    #cv2.imwrite('result.jpg', r.plot())

##**複数画像**

In [ ]:
from ultralytics import RTDETR
import cv2
import os

# 出力ディレクトリの作成
output_dir = '/content/video/output'
os.makedirs(output_dir, exist_ok=True)

# モデルは既にロード済みと仮定
try:
    model
except NameError:
    model = RTDETR(r"/content/drive/MyDrive/Deep_learning/ROP_AI_project/models/rtdetr-l-1700_1703.pt")

# 入力動画のパス
video_path = r'/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_movie/IMG_1623_16cm(福嶋）.mp4'
output_path = os.path.join(output_dir, 'result.mp4')

# 入力動画を開く
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: 動画ファイル '{video_path}' を開けませんでした")
    exit()

# 動画の基本情報を取得
target_width = 224
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# アスペクト比を保持してリサイズ
aspect_ratio = original_width / original_height
target_height = int(target_width / aspect_ratio)

# 出力用のビデオライター設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))

try:
    frame_count = 0
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # フレーム処理の進捗を表示
        frame_count += 1
        if frame_count % 10 == 0:  # 10フレームごとに進捗を表示
            print(f"Processing: {frame_count}/{total_frames} frames ({(frame_count/total_frames*100):.1f}%)")

        # リサイズ
        frame = cv2.resize(frame, (target_width, target_height))

        # 推論実行
        results = model(frame)

        # 結果を描画
        annotated_frame = results[0].plot()

        # フレームを書き出し
        out.write(annotated_frame)

except KeyboardInterrupt:
    print("\n処理が中断されました")
finally:
    # リソースの解放
    cap.release()
    out.release()

print(f"\n処理が完了しました。\n保存先: {output_path}")

#**Sample YOLO training**

```
最終形
YOLO_training/
├── images/
│   ├── train/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   │
│   └── val/
│       ├── img1.jpg
│       ├── img2.jpg
│       └── ...
│
├── labels/
│   ├── train/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
│   │
│   └── val/
│       ├── img1.txt
│       ├── img2.txt
│       └── ...
│
└── dataset.yaml


dataset.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: images/train
val: images/valid

# number of classes
nc: 1

# class names:
  0: Lens
  ```

In [ ]:
from ultralytics import RTDETR
import cv2
import os
from pathlib import Path

def export_yolo_annotations(image_dir, output_dir, model_path):
    # モデルのロード
    model = RTDETR(model_path)

    # ディレクトリの設定
    image_dir = Path(image_dir)
    output_dir = Path(output_dir)

    # 出力ディレクトリが存在しない場合は作成
    output_dir.mkdir(parents=True, exist_ok=True)

    # 画像ファイルの取得（.jpg, .png, .jpeg）
    image_files = []
    for ext in ['.jpg', '.jpeg', '.png']:
        image_files.extend(list(image_dir.glob(f'*{ext}')))

    print(f"処理する画像ファイル数: {len(image_files)}")

    for img_path in image_files:
        # 画像の読み込み
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"警告: {img_path}を読み込めませんでした")
            continue

        # 推論実行
        results = model(img)

        # アノテーションファイルのパス設定（出力ディレクトリに保存）
        txt_path = output_dir / f"{img_path.stem}.txt"

        # バウンディングボックスの取得
        boxes = results[0].boxes

        # YOLOフォーマットでアノテーションを書き出し
        with open(txt_path, 'w') as f:
            for box in boxes:
                # クラスID、正規化された中心x、中心y、幅、高さを取得
                cls = int(box.cls)
                x, y, w, h = box.xywhn[0].tolist()  # 正規化された座標を使用

                # YOLOフォーマットで書き出し
                f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

        print(f"保存完了: {txt_path}")

# モデルパスとデータディレクトリの設定
model_path = "/content/drive/MyDrive/Deep_learning/ROP_AI_project/models/rtdetr-l-1700_1703.pt"
images_dir = "/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_images/IMG_1703"
output_dir = "/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_annotations/IMG_1703"

# アノテーションの書き出し実行
export_yolo_annotations(images_dir, output_dir, model_path)

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def create_directory_structure(base_path):
    """指定されたパスに必要なディレクトリ構造を作成する"""
    directories = [
        os.path.join(base_path, 'images', 'train'),
        os.path.join(base_path, 'images', 'val'),
        os.path.join(base_path, 'labels', 'train'),
        os.path.join(base_path, 'labels', 'val')
    ]

    for directory in directories:
        os.makedirs(directory, exist_ok=True)

def copy_files(src_images, src_labels, dest_base, files, subset='train'):
    """ファイルを指定された場所にコピーする"""
    for filename in tqdm(files, desc=f'Copying {subset} files'):
        src_img = os.path.join(src_images, filename + '.jpg')
        dst_img = os.path.join(dest_base, 'images', subset, filename + '.jpg')

        src_txt = os.path.join(src_labels, filename + '.txt')
        dst_txt = os.path.join(dest_base, 'labels', subset, filename + '.txt')

        if os.path.exists(src_img) and os.path.exists(src_txt):
            shutil.copy2(src_img, dst_img)
            shutil.copy2(src_txt, dst_txt)

def organize_dataset(src_images, src_labels, dest_base, train_size=0.8, random_state=42):
    """データセットを整理し、train/valに分割する"""
    print('Creating directory structure...')
    create_directory_structure(dest_base)

    # 画像ファイルの一覧を取得（.jpgを除いたベース名）
    image_files = [os.path.splitext(f)[0] for f in os.listdir(src_images) if f.endswith('.jpg')]

    # train/valに分割
    train_files, val_files = train_test_split(
        image_files,
        train_size=train_size,
        random_state=random_state
    )

    # ファイルのコピー
    copy_files(src_images, src_labels, dest_base, train_files, 'train')
    copy_files(src_images, src_labels, dest_base, val_files, 'val')

    # 結果の表示
    print(f'\nDataset organization complete:')
    print(f'Training set: {len(train_files)} images')
    print(f'Validation set: {len(val_files)} images')

# メイン処理
if __name__ == "__main__":
    # パスの設定
    src_images = "/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_images/IMG_1703"
    src_labels = "/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_annotations/IMG_1703"
    dest_base = "/content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training"

    # データセットの整理を実行
    organize_dataset(src_images, src_labels, dest_base)

Creating directory structure...


Copying val files: 100%|██████████| 178/178 [00:05<00:00, 31.81it/s]


Dataset organization complete:
Training set: 710 images
Validation set: 178 images


In [ ]:
# YAMLファイルの書き出し

import yaml
import os

data = {
    'path': '/content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training',  # データセットのルートディレクトリ
    'train': 'images/train',  # 訓練用画像のディレクトリ
    'val': 'images/val',      # 検証用画像のディレクトリ
    'nc': 1,
    'names': {
        0: 'Lens',
    }
}

yaml_path = '/content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training/data.yaml'

# dataディレクトリが存在しない場合は作成
os.makedirs('data', exist_ok=True)

# ファイルの存在確認
if os.path.exists(yaml_path):
    print(f"既存の {yaml_path} を上書きします。")
else:
    print(f"新規に {yaml_path} を作成します。")

# YAMLファイルの書き出し（既存ファイルは上書き）
with open(yaml_path, 'w', encoding='utf-8') as f:
    yaml.safe_dump(data, f, sort_keys=False, allow_unicode=True)

print("処理が完了しました。")

既存の /content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training/data.yaml を上書きします。
処理が完了しました。


In [ ]:
### Train YOLO11 ###
from ultralytics import RTDETR
from ultralytics import YOLO
import wget

# YOLOモデルのダウンロードと初期化
try:
    #model = RTDETR('rtdetr-l.pt')
    model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
except:
    #wget.download('https://github.com/ultralytics/assets/releases/download/v0.0.0/rtdetr-l.pt')
    #model = RTDETR('rtdetr-l.pt')
    wget.download('https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11l.pt')
    model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
    # wget.download('https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m-seg.pt')
    # model = YOLO(r"C:\Users\ykita\ROP_AI_project\runs\segment\train9\weights\last.pt")  # load a pretrained model (recommended for training)

# Fine-tune
results = model.train(data='/content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training/data.yaml', epochs=1000, patience=100, batch=8, imgsz=640, device=0, flipud=0.5, fliplr=0.5)


Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training/data.yaml, epochs=1000, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fal

train: Scanning /content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training/labels/train.cache... 710 images, 0 backgrounds, 0 corrupt: 100%|██████████| 710/710 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training/labels/val.cache... 178 images, 0 backgrounds, 0 corrupt: 100%|██████████| 178/178 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        178        178          1          1      0.995      0.911



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     6/1000      1.38G     0.4454     0.4963     0.9011         19        640:  94%|█████████▍| 84/89 [00:28<00:01,  2.93it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-af9a32532af0>", line 19, in <cell line: 0>
    results = model.train(data='/content/drive/MyDrive/Deep_learning/ROP_AI_project/YOLO_training/data.yaml', epochs=1000, patience=100, batch=8, imgsz=640, device=0, flipud=0.5, fliplr=0.5)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/model.py", line 809, in train
    self.trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 208, in train
    self._do_train(world_size)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 381, in _do_train


TypeError: object of type 'NoneType' has no len()

# **Create dataset for CVAT**

階層構造
```
archive.zip/
   ├── data.yaml  # configuration file
   ├── train.txt  # list of train subset image paths
   │
   ├── images/
   │   ├── train/  # directory with images for train subset
   │   │    ├── image1.jpg
   │   │    ├── image2.jpg
   │   │    ├── image3.jpg
   │   │    └── ...
   ├── labels/
   │   ├── train/  # directory with annotations for train subset
   │   │    ├── image1.txt
   │   │    ├── image2.txt
   │   │    ├── image3.txt
   │   │    └── ...

```

***.txt (ラベル)
```
0 0.260745 0.535950 0.218015 0.030600
1 0.760795 0.533850 0.213035 0.043800
```

daya.yaml
```
names:
  0: Right_eye
  1: Left_eye
path: .
train: train.txt
```

train.txt
```
data/images/train/***.jpg
data/images/train/***.jpg
...(pathの羅列。拡張子を合わせる)
```
※ラベルや画像と対応を合わせること



In [ ]:
import os
import glob
import shutil
from tqdm import tqdm
from ultralytics import RTDETR

def create_yolo_dataset(orig_image_dir, output_dir, model_path):
    """
    YOLOデータセットの構造を作成する

    Args:
        orig_image_dir: 元の画像が存在するディレクトリ
        output_dir: 出力先のディレクトリ
        model_path: RTDETRモデルのパス
    """
    # 進捗バーの設定
    progress = tqdm(
        total=4,
        desc="データセット作成",
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
    )

    # 必要なディレクトリを作成
    os.makedirs(os.path.join(output_dir, "labels", "train"), exist_ok=True)
    progress.update(1)

    # data.yamlの作成
    yaml_content = """names:
  0: Right_eye
  1: Left_eye
path: .
train: train.txt"""

    with open(os.path.join(output_dir, "data.yaml"), 'w', encoding='utf-8') as f:
        f.write(yaml_content)
    progress.update(1)

    # モデルのロード
    try:
        model = RTDETR(model_path)
    except Exception as e:
        print(f"モデルのロードに失敗: {e}")
        return

    # 画像ファイルのリストを取得
    image_files = []
    extensions = ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG', '*.tif', '*.TIF', '*.tiff', '*.TIFF']

    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(orig_image_dir, ext)))

    total_files = len(image_files)
    print(f"\n合計 {total_files} 個のファイルが見つかりました")

    # 推論とラベル生成の進捗バー
    inference_progress = tqdm(
        total=total_files,
        desc="推論とラベル生成",
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
    )

    # train.txtの作成とラベルファイルの生成
    with open(os.path.join(output_dir, "train.txt"), 'w', encoding='utf-8') as f:
        for image_path in image_files:
            # 元の拡張子を保持
            basename = os.path.basename(image_path)
            name_without_ext = os.path.splitext(basename)[0]

            # 推論を実行
            results = model(image_path, save=False)

            # ラベルファイルのパス（train/配下に.txt）
            label_path = os.path.join(output_dir, "labels", "train", f"{name_without_ext}.txt")

            # YOLO形式の推論結果をテキストファイルに書き込み
            with open(label_path, 'w', encoding='utf-8') as lf:
                for r in results:
                    boxes = r.boxes
                    for box in boxes:
                        cls_id = int(box.cls[0])
                        x_center, y_center, width, height = box.xywhn[0]
                        line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
                        lf.write(line)

            # train.txtにYOLOフォーマットのパスを書き込み
            yolo_path = f"data/images/train/{basename}"
            f.write(f"{yolo_path}\n")
            inference_progress.update(1)

    inference_progress.close()
    progress.update(1)

    # ZIP圧縮
    print("\nZIP圧縮を開始します...")
    zip_path = output_dir + ".zip"
    total = sum(len(files) for _, _, files in os.walk(output_dir))

    with tqdm(total=total, desc="ZIP圧縮", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]') as pbar:
        for root, _, files in os.walk(output_dir):
            for file in files:
                pbar.update(1)
        shutil.make_archive(os.path.splitext(zip_path)[0], 'zip', output_dir)

    progress.update(1)
    progress.close()
    print(f"\nフォルダを {zip_path} に圧縮しました。")

# 使用例
if __name__ == "__main__":
    orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
    output_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"

    create_yolo_dataset(orig_image_dir, output_dir, model_path)

データセット作成:   0%|          | 0/4 [00:00<?]


合計 1390 個のファイルが見つかりました



image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_024df9c564e854ac63c356338fae68d7e8ebd20fafde2856d07f854576d25bec.JPG: 640x640 1 Right_eye, 1 Left_eye, 717.6ms
Speed: 7.0ms preprocess, 717.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_11ecdb1214665a16b19b0289b20c71d2c5a5a325f974312963bd044ddd10932f.JPG: 640x640 1 Right_eye, 1 Left_eye, 702.0ms
Speed: 4.0ms preprocess, 702.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_50a5a27bf14844c0973f0c274e935b2766ef37d438492d264e490a1e2dcaf11b.JPG: 640x640 1 Right_eye, 1 Left_eye, 705.5ms
Speed: 6.0ms preprocess, 705.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


...

データセット作成:  75%|███████▌  | 3/4 [17:31<05:50]


ZIP圧縮を開始します...


データセット作成: 100%|██████████| 4/4 [17:34<00:00]


フォルダを C:\Users\ykita\FacePhoto_instance\inference\201-295\archive.zip に圧縮しました。


: 

In [ ]:
import os
import glob
from ultralytics import RTDETR
from tqdm import tqdm  # tqdmライブラリをインポート

# 1) 画像のフォルダを指定する
orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
# 2) 出力先のフォルダを指定する。フォルダがなければ作成する
dst_parent_dir= r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive" #####
dst_label_dir = os.path.join(dst_parent_dir, "labels", "train")
os.makedirs(dst_label_dir, exist_ok=True)

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"
    model = RTDETR(model_path)

# 処理対象とする画像の拡張子
image_extensions = [
    '.jpg', '.jpeg', '.JPG', '.JPEG',
    '.png', '.PNG',
    '.tif', '.tiff', '.TIF', '.TIFF'
]

# 指定フォルダ配下の該当画像ファイルをすべて取得
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(orig_image_dir, f'*{ext}')))

# tqdmで進捗状況を表示しながらループ
for image_path in tqdm(image_files, desc='推論中', unit='枚'):
    # 推論を実行
    results = model(image_path, save=False)

    # 画像と同じファイル名（拡張子を除く）でテキストファイル出力
    basename = os.path.splitext(os.path.basename(image_path))[0]
    save_txt_path = os.path.join(dst_label_dir, f"{basename}.txt")

    # YOLO形式の推論結果をテキストファイルに書き込み
    with open(save_txt_path, 'w', encoding='utf-8') as f:
        for r in results:
            boxes = r.boxes  # YOLOv8系の出力
            for box in boxes:
                cls_id = int(box.cls[0])
                x_center, y_center, width, height = box.xywhn[0]
                # YOLO形式: class x_center y_center width height（小数点以下6桁）
                line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
                f.write(line)

print("すべての画像に対する推論が完了し、YOLO形式でラベルを出力しました。")


## **CVAT仕様のフォルダ構成に**

In [ ]:
import os
import glob

# 現在のlabelsフォルダのパス
orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
dst_parent_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"


# yamlとimg_listのパスを定義
data_yaml_path = os.path.join(dst_parent_dir, "data.yaml")
img_list_path = os.path.join(dst_parent_dir, "train.txt")

# data.yaml を書き込み
with open(data_yaml_path, 'w', encoding='utf-8') as f:
    f.write("names:\n")
    f.write("  0: Right_eye\n")
    f.write("  1: Left_eye\n")
    f.write("path: .\n")
    f.write("train: train.txt\n")

# labels_dir配下の .txt ファイルから、img_list.txt を作成
txt_files = glob.glob(os.path.join(dst_label_dir, "*.txt"))

with open(img_list_path, 'w', encoding='utf-8') as f:
    for txt_file in txt_files:
        basename = os.path.splitext(os.path.basename(txt_file))[0]
        f.write(f"data/images/train/{basename}.jpg\n")

print("data.yaml と img_list.txt が、labelsフォルダと同じ階層に作成されました。")

data.yaml と img_list.txt が、labelsフォルダと同じ階層に作成されました。


In [ ]:
import shutil

#dst_parent_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"
# dst_parent_dirをzip圧縮
zip_path = dst_parent_dir + ".zip"
shutil.make_archive(dst_parent_dir, 'zip', dst_parent_dir)

print(f"フォルダを {zip_path} に圧縮しました。")


フォルダを C:\Users\ykita\FacePhoto_instance\inference\201-295\archive.zip に圧縮しました。
